# Geodatenanalyse 1

## Übung 9: Statistisches Testen - Lösung

Nun wollen wir einige Hypothesen mit Hilfe von verschiedenen statistischen Tests an den Grundwasserdaten aus Karlsruhe testen. 

Lest dazu zunächst wieder den Datensatz ("Data_GW_KA.csv" oder "Data_GW_KA.xslx") in Python ein, und unterteilt ihn in die Daten aus dem Bereich des Waldes und der Stadt.

In [33]:
# [1]
import pandas as pd

data = pd.read_excel("Data_GW_KA.xlsx")

data_urban = data.loc[data.Flaechennutzung == 1,:]
data_forest = data.loc[data.Flaechennutzung == 2,:]

### Hypothese 1: "Die durchschnittliche Grundwassertemperatur im Wald beträgt 11°C" 

Zum Testen dieser Hypothese eignet sich ein zwei-seitiger t-Test. Die Nullhypothese H<sub>0</sub> haben wir bereits definiert, die alternative Hypothese lautet folglich "Die durchschnittliche Temperatur im Wald beträgt nicht 11°C". 

Der t-Test ist ein parametrischer Test, d.h. für einen gültigen Test müssen die Stichproben normal verteilt sein. Testet daher zuerst ob die Messwerte der Grundwassertemperatur diese Bedingung erfüllen. In `scipy.stats` gibt es die Funktion `shapiro()`, die einen Shapiro-Wilk-Test auf Normalität durchführt. Diese Funktion hat einen Input, den zu testenden Datensatz, und zwei Outputs, die Test-Statistik und den p-Wert. 

Führt den Shapiro-Wilk-Test für die Grundwassertemperatur durch und lasst euch den p-Wert anzeigen. Würdet Ihr für ein Signifikanzniveau von $\alpha$=0.01 annehmen, dass die Werte normalverteilt sind? 


In [34]:
# [3]
from scipy.stats import shapiro
stat, p = shapiro(data_forest["GW_Temperatur_°C"])
print(p)

0.7650017738342285


Da der p-Wert deutlich größer ist als das Signifikanzniveau ($\alpha$=0.01), können wir die Nullhypothese, dass die Werte normalverteilt sind, annehmen. Also könnt Ihr mit dem t-test für eine Stichprobe fortfahren. 

In `scipy.stats` gibt es dafür die Funktion `ttest_1samp()`. Als Inputs benötigt diese Funktion den zu testenden Datensatz und den hypothetischen Mittelwert. Formuliert das Skript mit Hilfe einer `if else` Bedingung so, dass es Euch (zusätzlich zum p-Wert) bei einem resultierenden p-Wert von >0.01 "H0 annehmen", und ansonsten "H0 nicht annehmen" ausgibt. 


In [35]:
# [4]
from scipy import stats
stat, p = stats.ttest_1samp(data_forest["GW_Temperatur_°C"], 11)
print('stat=%.4f, p=%.4f' % (stat, p))
if p > 0.01:
	print('H0 annehmen')
else:
	print('H0 nicht annehmen')




stat=-6.1775, p=0.0005
H0 nicht annehmen


Wie habt Ihr Euch in Bezug auf die Nullhypothese von oben entschieden? Interessant ist natürlich auch zu sehen, wo der Mittelwerte der Probe eigentlich liegt. Berechnet diesen,und vergleicht ihn mit dem hypothetischen Wert. Was fällt auf? Woran könnte das liegen? 

Hinweis: Bestimmt neben dem Mittelwert auch die Anzahl an Messwerten in Eurem Datensatz. 

In [36]:
# [5]
import statistics
mean_GWT = statistics.mean(data_forest["GW_Temperatur_°C"])
n = len(data_forest)
print(mean_GWT, n)

10.69125 8


### Hypothese 2: "Brunnen in der Stadt haben eine niedrigere Sauerstoffsättigung als Brunnen im Hardtwald" 

Auch diese Nullhypothese können wir mit einem t-Test überprüfen. Da es sich um den Vergleich von zwei Datensätzen handelt, benötigen wir eine Funktion für einen two-sample t-Test. Muss dieser hier ein- oder zwei-seitig sein? 

Für einen gültigen two-sample t-Test müssen beiden Datensätze normalverteilt sein, und zusätzlich die gleiche Varianz haben. Testet zuerst die Hypothesen ob die Sauerstoffsättigung im Wald, sowie in der Stadt normalverteilt sind. 

In [37]:
# [6]
stat_f, p_f = shapiro(data_forest["Sauerstoff_mg/l"])
print(p_f)
stat_u, p_u = shapiro(data_urban["Sauerstoff_mg/l"])
print(p_u)

0.9679369330406189
0.1985752433538437


Zum Testen der Varianzen zweier Datensätze nimmt man häufig den F-Test. Dabei lautet die Null-Hypothese, dass die Varianzen gleich sind, und die alternative Hypothese, dass die Varianzen verschieden sind. 

Die Formel für die F-Statistik lautet: 

<img src="https://latex.codecogs.com/gif.latex?\hat{F}&space;=&space;\frac{s_{a}^{2}}{s_{b}^{2}}" title="\hat{F} = \frac{s_{a}^{2}}{s_{b}^{2}}" />

wobei die Datensätze *a* und *b* so gewählt werden müssen, das gilt: s<sub>a</sub><sup>2</sup> > s<sub>b</sub><sup>2</sup>. 

Berechnet zuerst die Varianzen der beiden Sauerstoff-Datensätze. Stellt für die Berechnung der F-Statistik ("F-Wert") sicher, dass das Skript die Größe der Varianzen korrekt berücksichtigt. 

In [38]:
# [7]
import statistics
var_urban = statistics.variance(data_urban["Sauerstoff_mg/l"])
var_forest = statistics.variance(data_forest["Sauerstoff_mg/l"])
if var_urban > var_forest:
    sa = var_urban
    sb = var_forest
else:
    sa = var_forest
    sb = var_urban
F = sa/sb
print (F)

1.7914795850111382


Für das Testen der Hypothese wird dieser berechnete F-Wert nun mit dem kritischen F-Wert der Fisher-Verteilung verglichen, und der dazugehörige p-Wert berechnet. Dieser kann über `1 - scipy.stats.f.cdf()` berechnet werden. Die Funktion `scipy.stats.f.cdf()` benötigt als Inputs den F-Wert, und jeweils die Anzahl der Freiheitsgrade in beiden Datensätzen *a* und *b*. Da die Varianzen anhand der einzelnen Datenpunkte berechnet werden, entspricht die Anzahl an Freiheitsgraden jeweils `n-1`.

Lasst Euch den p-Wert anzeigen und vergleicht ihn mit dem Signifikanzniveau von 0.01. Würdet Ihr die Nullhypothese ("Beide Varianzen sind gleich") annehmen?


In [39]:
# [8]
import scipy
dfn = data_forest["Sauerstoff_mg/l"].size-1
dfd = data_urban["Sauerstoff_mg/l"].size-1
p = 1-scipy.stats.f.cdf(F, dfn, dfd)
print(p)

0.12584668944869004


Da nun beide Voraussetzungen für den t-Test überprüft sind, könnt Ihr diesen nun anwenden. Eine Option dazu ist die Funktion `scipy.stats.ttest_ind()`. Als Input werden hier die beiden Datensätze *a* und *b*  benötigt, wobei mean(*a*) > mean(*b*). Als Output liefert die Funktion den T-Wert und den p-Wert. 

Rechnet die beiden Mittelwerte aus, stellt sicher, dass die Größenverhältnisse stimmen, und lasst Euch den T-Wert und den p-Wert ausgeben. Achtung: da die Hypothese einen ein-seitigen t-Test verlangt, müsst Ihr den resultierenden p-Wert noch durch 2 teilen. Sollte man die Nullhypothese hier annehmen?

In [46]:
# [9]
mean_oxy_forest = data_forest["Sauerstoff_mg/l"].mean()
mean_oxy_urban = data_urban["Sauerstoff_mg/l"].mean()

print(mean_oxy_forest, mean_oxy_urban)

# am besten über if-else Bedingung anhand des größeren Mittelwert lösen
if mean_oxy_forest > mean_oxy_urban: 
    t_value, p = stats.ttest_ind(data_forest["Sauerstoff_mg/l"], data_urban["Sauerstoff_mg/l"])

else: 
    t_value, p = stats.ttest_ind(data_urban["Sauerstoff_mg/l"], data_forest["Sauerstoff_mg/l"])

p = p*0.5
print (t_value, p)

7.93875 4.468387096774194
3.4696694607406515 0.0006702352381517308


Ähnlich wie für den F-Test gibt es auch beim t-Test die Möglichkeit den kritischen T-Wert zu berechnen und darüber den p-Wert zu bestimmen. Die Funktion dazu lautet `scipy.stats.t.ppf()`, und benötigt als Input den Wert "1-Signifikanzniveau/2" sowie die Summe der Freiheitsgrade. 

In [48]:
# [10]
df = dfn + dfd 
t_crit = scipy.stats.t.ppf(1-0.05/2, df)
print(t_crit)

2.0261924630291093


Wenn der berechnete T-Wert größer als der kritische T-Wert ist, sollte die Nullhypothese abgelehnt werden. Stimmt das Ergebnis hier mit dem oben überein? 

### Was tun wenn keine Normalverteilung der Werte vorliegt?

Viele reale Datensätze sind nicht normalverteilt. Das nicht-parametrische Äquivalent zum t-Test ist der Mann-Whitney U-Test. Die Nullhypothese zu diesem Test lautet, dass die beiden Datensätze die gleiche Verteilung haben. Die alternative Hypothese lautet, dass die Verteilungen nicht gleich sind. 

Die passende Funktion dafür ist `scipy.stats.mannwhitneyu()`, und benötigt als Input die beiden zu vergleichenden Datensätze. 

Identifiziert über eine `for-Schleife` in dem Datensatz nicht-normalverteilte Parameter, testet diese (`1. if-Bedingung`) auf eine gleiche Verteilung in Wald und Stadtbereich, und lasst euch diejenigen (`2. if-Bedingung`) anzeigen die eine gleiche Verteilung haben (also bei denen die Null-Hypothese angenommen werden kann, p-Wert > Signifikanzniveau). 

Tipp: Baut Euren Code Schritt für Schritt auf und testet zwischendurch immer wieder die Funktionalität. Erst die Schleife mit dem Shapiro-Test, dann die erste If-Bedingung, dann die zweite usw.  

In [49]:
# [11]

for item in range(1, data.shape[1]-1): 
    #print(data[0, item])
    stat1, p1 = shapiro(pd.Series(data_forest.iloc[:,item]))
    stat2, p2 = shapiro(pd.Series(data_urban.iloc[:,item]))

    if p1 and p2 < 0.01:
        U_value, p_value = scipy.stats.mannwhitneyu(pd.Series(data_forest.iloc[:,item]), pd.Series(data_urban.iloc[:,item]))

        if p_value > 0.01:
            print (data.columns[item], U_value, p_value)
        

Phosphat_mg/l 69.5 0.058506478816411724
Detritus 113.0 0.6966670993087477
Sediment 127.5 0.9142043078871558
geologische_Einheit 85.0 0.14231142289670165
Anzahl_Arten 145.5 0.4520197594195483
Anzahl_Individuen 160.5 0.2103966930599971
Anteil_Crustaceen_% 162.0 0.18509761043690465


c:\Anaconda3\envs\GDA1\lib\site-packages\scipy\stats\morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


Falls Ihr nun noch Zeit habt, könnt Ihr Euch zum Üben beispielsweise weitere Hypothesen zu anderen Parametern überlegen und die testen. 

## Ende